In [ ]:
import numpy as np
from random import random, seed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as r2, mean_squared_error as mse
from sklearn.utils import resample
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps):

    """ computes the bias-variance decomposition of MSE through boostrapping """

    z_pred = np.empty((len(z_test), n_boostraps))
    for i in range(n_boostraps):
        data_, z_ = resample(data_train, z_train)
        model.fit(data_,z_)
        z_pred[:,i] = np.squeeze(model.predict(data_test)).T

    error = np.mean( np.mean((z_test - z_pred)**2, axis=1, keepdims=True) )
    bias = np.mean( (z_test - np.mean(z_pred, axis=1, keepdims=True))**2 )
    variance = np.mean( np.var(z_pred, axis=1, keepdims=True) )

    return error, bias, variance

In [ ]:
# create data set.
num_points = 100
x = np.linspace(-3, 3, num_points).reshape(-1, 1)
y = np.exp(-x**2) + 1.5 * np.exp(-(x-2)**2) + np.random.normal(0, 0.1, x.shape)
n_boostraps = 100

## Linear Regression

### OLS

In [ ]:
max_deg = 20
model = LinearRegression(fit_intercept=False)
error1 = np.zeros(max_deg)
bias1 = np.zeros(max_deg)
variance1 = np.zeros(max_deg)

# changing the degree of the polynomia
for deg in range(max_deg):

    poly = PolynomialFeatures(degree=deg+1)
    data = poly.fit_transform(x)

    data_train, data_test, z_train, z_test = train_test_split(data, y, test_size=0.2)

    error1[deg], bias1[deg], variance1[deg] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,max_deg+1), error1, label='MSE')
plt.plot(range(1,max_deg+1), bias1, label='Bias')
plt.plot(range(1,max_deg+1), variance1, label='Variance')
plt.title("Changing degree for OLS", fontsize=15)
plt.xlabel("Polynomial degree", fontsize=15)
plt.xticks(range(1,max_deg+1))
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
plt.savefig('Changing degree for OLS.png')
plt.show()

### Ridge

In [ ]:
deg = 16
lambda_v = np.logspace(-8,1,50)

poly = PolynomialFeatures(degree=deg+1)
data = poly.fit_transform(x)

data_train, data_test, z_train, z_test = train_test_split(data, y, test_size=0.2)

error2 = np.zeros(len(lambda_v))
bias2 = np.zeros(len(lambda_v))
variance2 = np.zeros(len(lambda_v))

for i in range(len(lambda_v)):

    lam = lambda_v[i]
    model = Ridge(lam,fit_intercept=False)

    error2[i], bias2[i], variance2[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(np.log(lambda_v), error2, label='MSE')
plt.plot(np.log(lambda_v), bias2, label='Bias')
plt.plot(np.log(lambda_v), variance2, label='Variance')
plt.title("Changing lambda for Ridge regression", fontsize=15)
plt.xlabel("log(lambda)", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
plt.savefig('Changing lambda for Ridge.png')
plt.show()

### Lasso

In [ ]:
deg = 12
lambda_v = np.logspace(-15,2,50)
#lambda_v=[]

poly = PolynomialFeatures(degree=deg+1)
data = poly.fit_transform(x)

data_train, data_test, z_train, z_test = train_test_split(data, y, test_size=0.2)

error3 = np.zeros(len(lambda_v))
bias3 = np.zeros(len(lambda_v))
variance3 = np.zeros(len(lambda_v))

for i in range(len(lambda_v)):

    lam = lambda_v[i]
    model = Lasso(lam,fit_intercept=False)

    error3[i], bias3[i], variance3[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(np.log(lambda_v), error3, label='MSE')
plt.plot(np.log(lambda_v), bias3, label='Bias')
plt.plot(np.log(lambda_v), variance3, label='Variance')
plt.title("Changing lambda for Lasso regression", fontsize=15)
plt.xlabel("log(lambda)", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
#plt.savefig('Changing lambda for Lasso regression.png')
plt.show()

## Tree-based methods

### Regression tree

In [ ]:
max_depth = 10

data_train, data_test, z_train, z_test = train_test_split(x,y, test_size=0.2)

model = DecisionTreeRegressor()

error4 = np.zeros(max_depth)
bias4 = np.zeros(max_depth)
variance4 = np.zeros(max_depth)

for i in range(max_depth):
    model.max_depth = i+1
    error4[i], bias4[i], variance4[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,max_depth+1), error4, label='MSE')
plt.plot(range(1,max_depth+1), bias4, label='Bias')
plt.plot(range(1,max_depth+1), variance4, label='Variance')
plt.title("Changing maximum depth for Regression Tree", fontsize=15)
plt.xlabel("Depth", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
plt.savefig('Changing maximum depth for Regression Tree')
plt.show()

### Random Forest

In [ ]:
num_points = 100

# Make data set.
x = np.linspace(-3, 3, num_points).reshape(-1, 1)
y = np.exp(-x**2) + 1.5 * np.exp(-(x-2)**2) + np.random.normal(0, 0.1, x.shape)

n_boostraps = 100
ntree_v = range(1,101,10)

data_train, data_test, z_train, z_test = train_test_split(x, y, test_size=0.2)

error5 = np.zeros(len(ntree_v))
bias5 = np.zeros(len(ntree_v))
variance5 = np.zeros(len(ntree_v))

for i in range(len(ntree_v)):

    ntree = ntree_v[i]
    model = RandomForestRegressor(n_estimators=ntree, verbose=False)
    error5[i], bias5[i], variance5[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(ntree_v, error5, label='MSE')
plt.plot(ntree_v, bias5, label='Bias')
plt.plot(ntree_v, variance5, label='Variance')
plt.title("Changing number of trees for Random Forest", fontsize=15)
plt.xlabel("Number of trees", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
#plt.savefig('Changing number of trees for Random Forest.png')
plt.show()

In [ ]:
m_v = range(1,13)
poly = PolynomialFeatures(12)
data = poly.fit_transform(x)

data_train, data_test, z_train, z_test = train_test_split(data, y, test_size=0.2)

error6 = np.zeros(len(m_v))
bias6 = np.zeros(len(m_v))
variance6 = np.zeros(len(m_v))

for i in range(len(m_v)):

    m = m_v[i]
    model = RandomForestRegressor(max_features=m)
    error6[i], bias6[i], variance6[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(m_v, error6, label='MSE')
plt.plot(m_v, bias6, label='Bias')
plt.plot(m_v, variance6, label='Variance')
plt.title("Changing number of predictors at each split for Random Forest", fontsize=15)
plt.xlabel("Number of predictors", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
#plt.savefig('Changing number of predictors at each split for Random Forest.png')
plt.show()

### AdaBoost

In [ ]:
ntree_v = range(1,101,10)

num_points = 50

# Make data set.
x = np.linspace(-3, 3, num_points).reshape(-1, 1)
y = np.exp(-x**2) + 1.5 * np.exp(-(x-2)**2) + np.random.normal(0, 0.1, x.shape)

n_boostraps = 20

data_train, data_test, z_train, z_test = train_test_split(x, y, test_size=0.2)

error7 = np.zeros(len(ntree_v))
bias7 = np.zeros(len(ntree_v))
variance7 = np.zeros(len(ntree_v))

for i in range(len(ntree_v)):

    ntree = ntree_v[i]
    model = AdaBoostRegressor(n_estimators=ntree)
    error7[i], bias7[i], variance7[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(ntree_v, error7, label='MSE')
plt.plot(ntree_v, bias7, label='Bias')
plt.plot(ntree_v, variance7, label='Variance')
plt.title("Changing number of trees for AdaBoost", fontsize=15)
plt.xlabel("Number of trees", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
#plt.savefig('Changing number of trees for AdaBoost.png')
plt.show()

## FFNN

In [ ]:
n_neurons_v = [1,2,5,10,20,30]

num_points = 50

# Make data set.
x = np.linspace(-3, 3, num_points).reshape(-1, 1)
y = np.exp(-x**2) + 1.5 * np.exp(-(x-2)**2) + np.random.normal(0, 0.1, x.shape)

n_boostraps = 20

data_train, data_test, z_train, z_test = train_test_split(x, y, test_size=0.2)

error8 = np.zeros(len(n_neurons_v))
bias8 = np.zeros(len(n_neurons_v))
variance8 = np.zeros(len(n_neurons_v))

for i in range(len(n_neurons_v)):
        
        n_neurons = n_neurons_v[i]
        model = MLPRegressor(hidden_layer_sizes=(n_neurons,),learning_rate_init=0.001, max_iter=500)
        error8[i], bias8[i], variance8[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(n_neurons_v, error8, label='MSE')
plt.plot(n_neurons_v, bias8, label='Bias')
plt.plot(n_neurons_v, variance8, label='Variance')
plt.title("Bias-Variance tradeoff with changing number of neurons for NN", fontsize=15)
plt.xlabel("Number of neurons", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
plt.savefig('Changing number of neurons for NN.png')
plt.show()

In [ ]:
n_layers_v = [1,2,3,4,5,6,7,8,9,10,11,12,13]

num_points = 50

# Make data set.
x = np.linspace(-3, 3, num_points).reshape(-1, 1)
y = np.exp(-x**2) + 1.5 * np.exp(-(x-2)**2) + np.random.normal(0, 0.1, x.shape)

n_boostraps = 20

data_train, data_test, z_train, z_test = train_test_split(x, y, test_size=0.2)

error9 = np.zeros(len(n_layers_v))
bias9 = np.zeros(len(n_layers_v))
variance9 = np.zeros(len(n_layers_v))

for i in range(len(n_layers_v)):
        
        n_layers = n_layers_v[i]
        model = MLPRegressor(hidden_layer_sizes=[20]*n_layers,learning_rate_init=0.001, max_iter=500)
        error9[i], bias9[i], variance9[i] = bootstrap_bias_variance_decomposition(data_train, data_test, z_train, z_test, model, n_boostraps)


In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(n_layers_v, error9, label='MSE')
plt.plot(n_layers_v, bias9, label='Bias')
plt.plot(n_layers_v, variance9, label='Variance')
plt.title("Bias-Variance tradeoff with changing number of layers for NN", fontsize=15)
plt.xlabel("Number of layers", fontsize=15)
plt.ylabel("Error", fontsize=15)
plt.legend(fontsize=15)
plt.savefig('Changing number of layers for NN.png')
plt.show()